# 구글 리뷰 - 크롤링 수집 작업 과정

구글 리뷰 크롤링 수집 작업은 다수의 데이터 수집을 하면서 발생한 문제들을 단계적으로 해결하며 발전했습니다.

### *1. 초기 버전[Jupyter Notebook]: Selenium을 이용한 단순 스크롤 및 리뷰 수집*
- 문제점: 스크롤을 할 때마다 이전에 로드된 리뷰가 중복 수집되는 문제 발생.

### *2. v2[Jupyter Notebook]: 선(先) 전체 스크롤, 후(後) 일괄 수집 방식으로 변경*
- 개선점: 중복 수집 문제 해결.
- 새로운 문제점: 수집 데이터량이 많아서 스크롤을 몇번 해야 지정한 날짜까지 데이터가 수집되는지를 확인 불가.
   
### *3. v3[Jupyter Notebook]: 목표 날짜 도달 시 스크롤을 중단하는 로직 추가*
- 개선점: 수집 시간 단축.
- 새로운 문제점: 수집 데이터량이 많을 경우, 일괄 수집 전 메모리 초과로 프로세스가 중단되는 안정성 문제 발견.

### *4. v4(크롤링 final 버전)[Jupyter Notebook]: 중간 저장 기능 추가*
- 개선점: 일정량의 스크롤마다 수집된 데이터를 파일에 중간 저장하여, 프로세스가 중단되더라도 재개가 가능하도록 안정성 확보.
- 새로운 문제점: v3보다 보다 많은 데이터 수집은 가능하지만, ui를 그리면서 작업이 진행되기 때문에 메모리 초과 문제가 완전히 해결되지 않음.
   
### <b>5. final 버전(방식 변경): google-play-scraper 라이브러리 도입</b>
- 근본적인 해결: Selenium 기반 UI 자동화의 근본적인 한계(속도, 안정성)를 인지하고, API 방식으로 데이터를 직접 요청하는 google-play-scraper로 방법론을 전면 교체.
- 최종 결과: 메모리 문제와 속도 문제를 근본적으로 해결하여, 안정적이고 빠른 대규모 리뷰 수집 시스템을 완성.

In [ ]:
# 구글 플레이 리뷰 수집을 위한 기능 설치
!pip install google-play-scraper

In [ ]:
from google_play_scraper import Sort, reviews
import pandas as pd
from datetime import datetime

app_id = 'com.kakao.talk'

# --- setting ---
start_date = datetime(2025, 09, 22)
end_date = datetime(2025, 11, 18, 23, 59, 59) # 종료일 전체를 포함하도록 시간까지 포함

collected_reviews = []
continuation_token = None
stop_collecting = False # while 문을 반복하기 위한 변수 역할로 변수명의 의미를 살리기 위해 최초 False로 설정 > not stop_collecting : 이라고 사용하여 True로 전환해 사용하는 것

print(f"'{app_id}' 앱의 리뷰를 '{start_date.date()}'까지 수집합니다...")

while not stop_collecting:
    # reviews 함수 실행 후 리뷰 리스트(result)와 다음 페이지가 어디인지에 대한 값(continuation_token)을 가져오기 때문에 result와 continuation_token 의 2개 변수로 가져온 값을 저장하는 것
    result, continuation_token = reviews(
        app_id,
        lang='ko',
        country='kr',
        sort=Sort.NEWEST,
        count=200,
        continuation_token=continuation_token
    )

    if not result:
        print("더 이상 가져올 리뷰가 없어 수집을 종료합니다.")
        break

    batch_ended = False
    for review in result:
        # 구글 스크래퍼를 통해 리뷰 데이터를 가져올 때 날짜 및 시간 데이터는 "at" 컬럼으로  전달되는 것이 지정되어 있음
        # 현재 확인 중인 리뷰의 날짜[at]가 시작 날짜 전이면
        if review['at'] < start_date:
            print(f"-> 목표 날짜({start_date.date()})에 도달하여 수집을 중단합니다.")
            stop_collecting = True # 바깥쪽 while 루프를 종료
            batch_ended = True    # 안쪽 for 루프를 종료
            break # for 루프 탈출

        # 현재 확인 중인 리뷰의 날짜가 우리가 원하는 기간 안에 있으면, 리스트에 추가
        if start_date <= review['at'] <= end_date:
            collected_reviews.append(review)
  # ----------------------------------------------------

    if batch_ended:
        break # while 루프 탈출

    print(f"페이지 수집 완료. 현재까지 수집된 총 리뷰 수: {len(collected_reviews)}개, 마지막 리뷰 날짜: {result[-1]['at'].date()}")
    # [-1] : 가장 마지막 데이터
    # date() : 시분초를 제외한 날짜(년-월-일) 형식으로 제공

if collected_reviews:
    df_reviews = pd.DataFrame(collected_reviews)
    print("\n" + "="*50)
    print(f"총 {len(df_reviews)}개의 리뷰를 성공적으로 수집했습니다.")
    print("="*50)
    display(df_reviews[['userName', 'score', 'at', 'content']])
else:
    print("\n해당 기간 동안 수집된 리뷰가 없습니다.")

In [ ]:
# 코랩에서 파일을 다운로드하기 위한 라이브러리 호출
from google.colab import files

# 파일 이름 설정
csv_filename = '251118_kakao_reviews_final.csv'

print(f"총 {len(df_reviews)}개의 리뷰를 '{csv_filename}' 파일로 저장합니다...")

# 데이터프레임을 CSV 파일로 저장 (한글 깨짐 방지를 위해 'utf-8-sig' 사용)
df_reviews.to_csv(csv_filename, encoding='utf-8-sig', index=False)

print("파일 저장 완료. 다운로드를 시작합니다.")

# 코랩 환경에서 내 PC로 파일 다운로드
files.download(csv_filename)